In [1]:
import sys

sys.path.append('../')

In [2]:
import pandas as pd
import numpy as np

In [3]:
from datetime import datetime, timedelta

In [4]:
today_d = datetime.today()
start_d = today_d - timedelta(days = 40)

In [5]:
query_start = start_d.strftime('%Y-%m-%d')
query_end = today_d.strftime('%Y-%m-%d')

In [6]:
query_start, query_end

('2021-08-12', '2021-09-21')

In [7]:
tic_list = ['DOGE-USD']

tech_indicators = ['cci_30',
 'rsi_30',
 'rsi_14',
 'rsi_6',
 'dx_30', 
 'dx_14']

### Daily configuration

In [8]:
coin_balance = np.array([122.0], dtype = float)
cash_balance = 39.3653
model_file = './CryptoModel/doge_model_260.pkl'

### Pre-train setup

In [9]:
reward_on_value = True
lookback_n = 3

config_max_step = 15

if reward_on_value:
    reward_scaling = 2 ** -10
else:
    reward_scaling = 2 ** -5

### Query last 40 days price

In [10]:
from neo_finrl.data_processors.processor_yahoofinance import YahooFinanceProcessor

In [11]:
data_downloader = YahooFinanceProcessor()

In [12]:
stock_history_df = data_downloader.download_data(query_start, query_end, tic_list, '1D')

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (41, 9)


In [13]:
data_downloader.time_interval = '1D'
stock_history_df = data_downloader.clean_data(stock_history_df)

Clean data for DOGE-USD
Data clean for DOGE-USD is finished.
Data clean all finished!


In [14]:
stock_history_df = data_downloader.add_technical_indicator(stock_history_df, tech_indicators)

In [15]:
stock_history_df.tail(2)

,time,open,high,low,close,adjcp,volume,tic,cci_30,rsi_30,rsi_14,rsi_6,dx_30,dx_14
39,2021-09-19,0.241281,0.241285,0.231337,0.233142,0.233142,8.927640e+08,DOGE-USD,-104.174005,41.558349,36.978739,30.578275,29.929329,28.748286
40,2021-09-20,0.233161,0.233606,0.200022,0.207071,0.207071,2.244004e+09,DOGE-USD,-148.601331,37.129793,29.955212,17.497940,44.655428,56.551056


In [16]:
#stock_history_df.to_csv('./DOGE_px_20210918.csv', index = False)

### Model run

In [17]:
from test_env.single_crypto_env import CryptoTradingEnv

from stable_baselines3 import PPO, DDPG
from stable_baselines3.common.vec_env import DummyVecEnv, VecCheckNan, VecNormalize
from stable_baselines3.common.logger import configure

In [18]:
tmp_path = "./tmp/sb3_log/"
# set up logger
new_logger = configure(tmp_path, ["stdout", "csv"])

Logging to ./tmp/sb3_log/


In [19]:
def modelRun(start_idx, px_df, input_amount, input_stocks, last_model):
    def tradable_size(env, x):
            return (x / env.min_stock_rate).astype(int) * env.min_stock_rate
    
    test_config = dict()

    test_config['price_array'] = px_df.iloc[:(start_idx + config_max_step)][['open', 'adjcp', 'low', 'high']].values
    test_config['tech_array'] = px_df.iloc[:(start_idx + config_max_step)][tech_indicators].values

    #randomly start day index for back testing
    test_config['if_sequence'] = True
    # disable random initial capital 
    test_config['if_randomV'] = False

    test_config['if_value'] = reward_on_value
    test_config['lookback_n'] = lookback_n

    max_step = min(config_max_step, px_df.shape[0] - start_idx) - 1
    
    print ('Run model from ', start_idx, ' to ', start_idx + max_step)
    
    test_env = CryptoTradingEnv(test_config, \
                            initial_capital=input_amount, \
                            max_step = max_step, \
                           initial_stocks = input_stocks, 
                           reward_scaling = reward_scaling, \
                            start_idx = start_idx)
    state = test_env.reset()
    
    print (state)
    
    #test_model = PPO.load(cwd)
    test_model = DDPG.load(last_model)
    test_model = test_model.policy.eval()
    
    action = test_model.predict(state)[0]   
    
    # actions -> percentage of stock or cash
    # add clip at 0.9
    actions_v = action[0] * 0.9
    
    if actions_v == np.nan:
        actions_v = 0.0
        
    order_px = (test_env.price_ary[test_env.day + test_env.run_index, 0] + \
                    test_env.price_ary[test_env.day + test_env.run_index, 1])/2.0
        
    print ('Action value: ', actions_v)
        
    if actions_v > 0:
        if test_env.amount * actions_v > test_env.buy_min_value:
            buy_num_shares = tradable_size(test_env, (test_env.amount * actions_v/order_px)/(1 + test_env.buy_cost_pct))
            print (f'Buy {buy_num_shares} at price {order_px}')
        else:
            print ('No buy action today')
            
    if actions_v < 0:
        sell_num_shares = tradable_size(test_env, test_env.stocks[0] * (-1.0) * actions_v)
        # no short 
        sell_num_shares = min(sell_num_shares, test_env.stocks[0])
        print (f'Sell {sell_num_shares} at price {order_px}')
        
    print ("\n")
    print ("[!!Warning!!] check current MKT price for better deal!!")
    
    return -1

In [20]:
#stock_history_df = pd.read_csv('./DOGE_px_20210918.csv')

In [21]:
modelRun(stock_history_df.shape[0]-1, 
         stock_history_df, 
         cash_balance,
         coin_balance, 
         model_file)

Run model from  40  to  40
initial stock: [122.] inital amount:  39.3653
initial asset:  64.62796288757323
[ 0.6091063   0.95971203  0.96554399  0.95291197  0.98931199  0.96512401
  0.93256801  0.92534798  0.96513999  0.93264401  0.82828403  0.80008799
  0.93442398 -1.16094792  0.29007649  0.23402509  0.13670266  0.34887055
  0.44180512]
Action value:  -0.7947917461395264
Sell 96.9645 at price 0.2201160043478012


[!!Warning!!] check current MKT price for better deal!!


-1